<a target="_blank" href="https://colab.research.google.com/github/ArtificialIntelligenceToolkit/aitk/blob/master/notebooks/Robotics/SeekLight.ipynb"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# A robot that seeks light

In this notebook we will experiment with a robot that seeks light in variety of different worlds from quite easy, where the light source is simply out in the open, to much harder, where the light source is hidden within an enclosed room. Even though the worlds are different, we will use the same controller to solve them.

In [1]:
%pip install aitk --quiet

In [2]:
from aitk.robots import *
from random import randrange

## Create a robot

The robot will have both left and right range sensors to detect obstacles and left and right light sensors to detect the light source.

In [3]:
robot = Scribbler(x=150,y=160,a=180)
robot.add_device(RangeSensor(position=(6,-6),width=57.3,max=20,name="left-ir"))
robot.add_device(RangeSensor(position=(6,6),width=57.3,max=20,name="right-ir"))
robot.add_device(LightSensor(position=(6,-6),name="left-light"))
robot.add_device(LightSensor(position=(6,6),name="right-light"))

## Create a controller to seek the light

On each time step the controller will check the status of the robots sensors in order to determine the best course of action:

* When the light readings are high encough, the robot will stop and report that it has found the light.
* When the range sensors are not detecting any obstacles, the robot will try to turn and move towards the light source, but if no light is detected it will move foward.
* When an obstacle is detected, the robot will start a timed turn to move continuously in the same direction away from the obstacle.

In [4]:
def seekLight(robot):
    left_ir = robot["left-ir"].get_distance()
    right_ir = robot["right-ir"].get_distance()
    left_light = robot["left-light"].get_brightness()
    right_light = robot["right-light"].get_brightness()
    total_light = left_light + right_light
    diff_light = left_light - right_light
    if total_light > 1.95:
        print("\nFound light!")
        return True
    elif left_ir == robot[0].get_max() and right_ir == robot[1].get_max() and \
        robot.state["timer"] == 0:
        if abs(diff_light) <= robot.state["light-diff"]:
            # move forward
            robot.move(0.5, 0)
        elif diff_light < 0:
            # light stronger to right
            robot.move(0.2, -0.1)
        else:
            # light stronger to left
            robot.move(0.2, 0.1)
    elif robot.state["timer"] > 0 and robot.state["timer"] < 5:
        # timer triggered, continue current rotation
        robot.state["timer"] += 1
    elif left_ir < robot[0].get_max():
        # obstacle on left, turn right, trigger timer
        robot.move(0.1, -0.4)
        robot.state["timer"] = 1
    elif right_ir < robot[1].get_max():
        # obstacle on right, turn left, trigger timer
        robot.move(0.1, 0.4)
        robot.state["timer"] = 1
    else:
        # reset timer to zero
        robot.state["timer"] = 0


In [5]:
def reset_robot():
  """
  Resets the robot to a new random location in the world and sets its
  state variables to their initial values.
  """
  robot.set_random_pose()
  robot.state["timer"] = 0
  robot.state["light-diff"] = 0.02

## Easiest World

In the easiest world, the light is out in the open and the robot will quickly sense the light and approach it.

In [6]:
world1 = World(width=200,height=200)
world1.add_bulb("yellow", 100, 90, 0, 50)
world1.add_robot(robot)
world1.watch()

Random seed set to: 6012431


Image(value=b'\xff\xd8\xff\xe0\x00\x10JFIF\x00\x01\x01\x00\x00\x01\x00\x01\x00\x00\xff\xdb\x00C\x00\x08\x06\x0…

In [7]:
reset_robot()
world1.update()
world1.seconds(120, [seekLight], real_time=True)

  0%|          | 0/1200 [00:00<?, ?it/s]


Found light!
Simulation stopped at: 00:00:10.60; speed 0.97 x real time


True

## Harder World

In this harder world the light is positioned next to a wall, so the robot must navigate the wall when approaching the light.


In [8]:
world2 = World(width=200,height=200)
world2.add_wall("blue", 50, 110, 150, 120)
world2.add_bulb("yellow", 100, 90, 0, 50)
world2.add_robot(robot)
world2.watch()

Random seed set to: 7612895


Image(value=b'\xff\xd8\xff\xe0\x00\x10JFIF\x00\x01\x01\x00\x00\x01\x00\x01\x00\x00\xff\xdb\x00C\x00\x08\x06\x0…

In [9]:
reset_robot()
world2.update()
world2.seconds(120, [seekLight], real_time=True)

  0%|          | 0/1200 [00:00<?, ?it/s]


Found light!
Simulation stopped at: 00:00:25.00; speed 0.98 x real time


True

## More Challenging World

In this more challenging world the light is enclosed in a corral, make it difficult for the robot to find the light, and when found to enter the corral to reach the light.

In [10]:
world3 = World(width=200,height=200)
world3.add_wall("blue", 50, 50, 60, 120)
world3.add_wall("blue", 50, 40, 80, 50)
world3.add_wall("blue", 140, 50, 150, 120)
world3.add_wall("blue", 120, 40, 150, 50)
world3.add_wall("blue", 50, 110, 150, 120)
world3.add_bulb("yellow", 100, 90, 0, 50)
world3.add_robot(robot)
world3.watch()

Random seed set to: 1586416


Image(value=b'\xff\xd8\xff\xe0\x00\x10JFIF\x00\x01\x01\x00\x00\x01\x00\x01\x00\x00\xff\xdb\x00C\x00\x08\x06\x0…

In [11]:
reset_robot()
world3.update()
world3.seconds(120, [seekLight], real_time=True)

  0%|          | 0/1200 [00:00<?, ?it/s]


Found light!
Simulation stopped at: 00:00:42.10; speed 0.98 x real time


True

In all three world the robot should be able to find the light source, however we would expect the average time it takes will increase with the complexity of the world.